In [1]:
# %%
import os

os.environ["TOKENIZERS_PARALLELISM"] = 'false'
os.environ["WANDB_DIR"] = '/projets/melodi/gsantoss/wandbt'

from cmatcher.owl_utils import *
from cmatcher.cqa_search import *
from cmatcher.eval_utils import *
from transformers import AutoTokenizer
import dill
import torch
import torch.optim as optim
import copy
import tqdm
from cmatcher.model import *
from tqdm.auto import tqdm
import random
import wandb

import argparse

from rdflib import Graph, URIRef, BNode
from rdflib.namespace import RDF, RDFS, OWL
from om.ont import get_namespace, get_n
from termcolor import colored
import os
from tqdm.auto import tqdm
from os import path
from cmatcher.epc import parser
from multiprocessing_on_dill import Pool
import torch



torch.manual_seed(0)
random.seed(0)

In [2]:
ontology_paths = {
    'edas.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/edas.owl',
    'ekaw.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/ekaw.owl',
    'confOf.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/confOf.owl',
    'conference.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/conference.owl',
    'cmt.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/cmt.owl',
}

cqa_path = '/projets/melodi/gsantoss/data/complex/CQAs'
entities_path = '/projets/melodi/gsantoss/data/complex/entities-cqas'

In [3]:
cqas = load_cqas(cqa_path)

In [4]:
print(cqas.keys())

In [103]:
class VirtualDocumentGenerator:
    
    def __init__(self, g):
        self.g = g


    def get_ns(self, e):
        if e is None:
            return 'None'
        return e.split('#')[0]
    
    def get_parents(self, e):    
        parent = self.g.value(e, RDFS.subClassOf)    
        if parent is None:
            return []
        
        return [parent] + self.get_parents(parent)
    
    def get_children(self, e):    
        children = list(self.g.subjects(RDFS.subClassOf, e))  
        return children 
        
    def get_outgoing_properties(self, e):
        op = []
        
        for s, p, o in self.g.triples((e, None, None)):
            if p in {RDF.type}:
                continue
            op.append((p, o))
        
        return op
    
    def get_incoming_properties(self, e):
        ip = []
        
        for s, p, o in self.g.triples((None, None, e)):
            if p in {RDF.first, RDF.rest}:
                continue
            ip.append((s, p))
        
        return ip
    
    
    
    def list_to_virtual_document(self, l):
        if len(l) == 0:
            return 'Empty'
        
        
        fl = []
        
        for e in l:
            if type(e) is tuple:
                fl.append(f'{e[0]} {e[1]}')
            else:
                fl.append(e)
        
        return ', '.join(fl)
    
    
    def to_virtual_document(self, e):
        
        if e is OWL.Thing:
    
            return f'''Description of the entity: {e}'''
        
        if type(e) is BNode:
                   
            return f'''Description of the entity: {e}'''
        elif self.get_ns(g.value(e, RDF.type)) == 'http://www.w3.org/2002/07/owl':
            
            parents = self.list_to_virtual_document(self.get_parents(e))
            children = self.list_to_virtual_document(self.get_children(e))
            outgoing_properties = self.list_to_virtual_document(self.get_outgoing_properties(e))
            incoming_properties = self.list_to_virtual_document(self.get_incoming_properties(e))
        
            return f'''Description of the entity: {e}.\n\tParents: {parents}.\n\tChildren: {children}.\n\tOutgoing properties: {outgoing_properties}.\n\tIncoming properties: {incoming_properties}.'''
        
        else:
            parentType = self.g.value(e, RDF.type)
            parents = self.list_to_virtual_document(self.get_parents(parentType))
            children = self.list_to_virtual_document(self.get_children(parentType))
            outgoing_properties = self.list_to_virtual_document(self.get_outgoing_properties(e))
            incoming_properties = self.list_to_virtual_document(self.get_incoming_properties(e))
            return f'''Description of the entity: {e}.\n\tInstance of: {parentType}.\n\tParents: {parents}.\n\tChildren: {children}.\n\tOutgoing properties: {outgoing_properties}.\n\tIncoming properties: {incoming_properties}.'''
    




for v in ontology_paths.values():
    g = Graph().parse(v) 
    dg = VirtualDocumentGenerator(g)
    for s in set(g.subjects()):
        if g.value(s, RDF.first) is not None:
            continue
        
        d = dg.to_virtual_document(s)
        print(d)
        print('-'*50)
    
    break

In [17]:
data = {}

graphs = {}

for p, d, fs in tqdm(list(os.walk(entities_path))):
    for f in fs:

        if f not in graphs:
            graphs[f] = Graph().parse(ontology_paths[f])

        g = graphs[f]
        cqa = p.split('/')[-1]
        if cqa not in data:
            data[cqa] = {}

        with open(path.join(p, f), 'r') as fl:
            text = fl.read()
            tree = parser.parse(text)
            ng = Graph()

            tn = parse_tree(tree, g, ng)
            
            
            for s, p, o in ng:
                print(s, p, o)
            
            
            

            
            raise Exception('hue')
